In [ ]:
!pip install transformers

In [1]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import numpy as np

import logging
logging.getLogger().setLevel(logging.CRITICAL)

import warnings
warnings.filterwarnings('ignore')

from transformers import AdamW, get_linear_schedule_with_warmup

In [2]:
from torch.utils.data import Dataset
from torch.utils.data import Dataset, DataLoader
import os
import json
import csv

In [3]:
def choose_from_top(probs, n=5):
    ind = np.argpartition(probs, -n)[-n:]
    top_prob = probs[ind]
    top_prob = top_prob / np.sum(top_prob) # Normalize
    choice = np.random.choice(n, 1, p = top_prob)
    token_id = ind[choice][0]
    return int(token_id)

In [4]:
class DistractorsDataset(Dataset):
    def __init__(self, dataset_path = 'trian.json'):
        super().__init__()

        with open('train.json') as json_file:
          train_data = json.load(json_file)

        self.input_list = []
        self.end_of_text_token = '<|endoftext|>'

        for line in train_data:
          dist1 = f"<question>{line['question']}<key>{line['correct_answer']}<distractor>{line['distractor1']}{self.end_of_text_token}"
          self.input_list.append(dist1)
          dist2 = f"<question>{line['question']}<key>{line['correct_answer']}<distractor>{line['distractor2']}{self.end_of_text_token}"
          self.input_list.append(dist2)
          dist3 = f"<question>{line['question']}<key>{line['correct_answer']}<distractor>{line['distractor3']}{self.end_of_text_token}"
          self.input_list.append(dist3)

    def __len__(self):
        return len(self.input_list)

    def __getitem__(self, item):
        return self.input_list[item]

In [5]:
dataset = DistractorsDataset()
distractors_loader = DataLoader(dataset, batch_size=1, shuffle=True)

In [6]:
BATCH_SIZE = 16
EPOCHS = 5
LEARNING_RATE = 3e-5
WARMUP_STEPS = 5000
MAX_SEQ_LEN = 400

device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

In [7]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium')
model = GPT2LMHeadModel.from_pretrained('gpt2-medium')
model = model.to(device)

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [8]:
torch.manual_seed(54321)
model = model.to(device)
model.train()
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=WARMUP_STEPS, num_training_steps = -1)
proc_seq_count = 0
sum_loss = 0.0
batch_count = 0

tmp_distractor_tens = None
models_folder = "trained_models"
if not os.path.exists(models_folder):
    os.mkdir(models_folder)

for epoch in range(EPOCHS):

    print(f"EPOCH {epoch} started" + '=' * 30)

    for idx, distractor in enumerate(distractors_loader):

        distractor_tens = torch.tensor(tokenizer.encode(distractor[0])).unsqueeze(0).to(device)
        if distractor_tens.size()[1] > MAX_SEQ_LEN:
            continue

        if not torch.is_tensor(tmp_distractor_tens):
            tmp_distractor_tens = distractor_tens
            continue
        else:
            if tmp_distractor_tens.size()[1] + distractor_tens.size()[1] > MAX_SEQ_LEN:
                work_distractor_tens = tmp_distractor_tens
                tmp_distractor_tens = distractor_tens
            else:

                tmp_distractor_tens = torch.cat([tmp_distractor_tens, distractor_tens[:,1:]], dim=1)
                continue


        outputs = model(work_distractor_tens, labels=work_distractor_tens)
        loss, logits = outputs[:2]
        loss.backward()
        sum_loss = sum_loss + loss.detach().data

        proc_seq_count = proc_seq_count + 1
        if proc_seq_count == BATCH_SIZE:
            proc_seq_count = 0
            batch_count += 1
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
            model.zero_grad()

        if batch_count == 100:
            print(f"sum loss {sum_loss}")
            batch_count = 0
            sum_loss = 0.0

    # Store the model after each epoch to compare the performance of them
    torch.save(model.state_dict(), os.path.join(models_folder, f"gpt2_medium_distractor_{epoch}.pt"))


EPOCH 0 started==============================
sum loss 5881.998046875
EPOCH 1 started==============================
sum loss 4938.43896484375
sum loss 4150.37646484375
EPOCH 2 started==============================
sum loss 3797.967529296875
sum loss 3651.997802734375
EPOCH 3 started==============================
sum loss 3550.4033203125
sum loss 3468.11328125
EPOCH 4 started==============================
sum loss 3390.6669921875


In [11]:
MODEL_EPOCH = 4

models_folder = "trained_models"

model_path = os.path.join(models_folder, f"gpt2_medium_distractor_{MODEL_EPOCH}.pt")
model.load_state_dict(torch.load(model_path))

distractors_output_file_path = f'generated_{MODEL_EPOCH}.distractors'

model.eval()

if os.path.exists(distractors_output_file_path):
    os.remove(distractors_output_file_path)

distractor_num = 0

questions = ['<question>What does a Geiger Counter measure?<key>Radiation<distractor>',
             '<question>What does a Geiger Counter measure?<key>Radiation<distractor>',
             '<question>What does a Geiger Counter measure?<key>Radiation<distractor>',
             '<question>A stable subatomic particle with a charge of negative electricity, found in all atoms and acting as the primary carrier of electricity in solids.<key>Electron<distractor>',
             '<question>A stable subatomic particle with a charge of negative electricity, found in all atoms and acting as the primary carrier of electricity in solids.<key>Electron<distractor>',
             '<question>A stable subatomic particle with a charge of negative electricity, found in all atoms and acting as the primary carrier of electricity in solids.<key>Electron<distractor>',
             '<question>What is the most abundant gas in the atmosphere of the Earth?<key>Nitrogen<distractor>',
             '<question>What is the most abundant gas in the atmosphere of the Earth?<key>Nitrogen<distractor>',
             '<question>What is the most abundant gas in the atmosphere of the Earth?<key>Nitrogen<distractor>'
 ]
with torch.no_grad():

        for question in questions:

            distractor_generation_finished = False

            cur_ids = torch.tensor(tokenizer.encode(question)).unsqueeze(0).to(device)

            for i in range(100):
                outputs = model(cur_ids, labels=cur_ids)
                loss, logits = outputs[:2]
                softmax_logits = torch.softmax(logits[0,-1], dim=0) #Take the first(from only one in this case) batch and the last predicted embedding
                if i < 3:
                    n = 20
                else:
                    n = 3
                next_token_id = choose_from_top(softmax_logits.to('cpu').numpy(), n=n) #Randomly(from the topN probability distribution) select the next word
                cur_ids = torch.cat([cur_ids, torch.ones((1,1)).long().to(device) * next_token_id], dim = 1) # Add the last word to the running sequence

                if next_token_id in tokenizer.encode('<|endoftext|>'):
                    distractor_generation_finished = True
                    break


            if distractor_generation_finished:

                distractor_num = distractor_num + 1

                output_list = list(cur_ids.squeeze().to('cpu').numpy())
                output_text = tokenizer.decode(output_list)

                with open('generated_distractors.txt', 'a') as f:
                    f.write(f"{output_text} \n\n")
